# MPD Payroll Investigation
---------
This project is going to discover the relationship between how much a policeman/woman is paid and any relevant factors.

In [ ]:
options(warn=-1)

## Import libraries

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load
# library(sjmisc)
library(magrittr)
library(maps) 
library(dplyr) 
library(DescTools)
library(naniar)
library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Data

In [ ]:
Boston = read_csv('../input/masscities/Boston.csv')
Brockton = read_csv('../input/masscities/Brockton.csv')
Cambridge = read_csv('../input/masscities/Cambridge.csv')
Lynn = read_csv('../input/masscities/Lynn.csv')
Springfield = read_csv('../input/masscities/Springfield.csv')

## EDA

### Data Briefing

In [ ]:
head(Boston, n=5)

In [ ]:
head(Brockton, n=5)

In [ ]:
head(Cambridge, n=5)

In [ ]:
head(Lynn, n=5)

In [ ]:
head(Springfield, n=5)

### Concatenation

In [ ]:
# drop the meaningless column
drops <- c("_type")
Boston <- Boston[ , !(names(Boston) %in% drops)]
Brockton <- Brockton[ , !(names(Brockton) %in% drops)]
Cambridge <- Cambridge[ , !(names(Cambridge) %in% drops)]
Lynn <- Lynn[ , !(names(Lynn) %in% drops)]
Springfield <- Springfield[ , !(names(Springfield) %in% drops)]

In [ ]:
data = rbind(rbind(rbind(rbind(Boston, Brockton), Cambridge), Lynn), Springfield)
colnames(data) <- c("Annual_Wage", "Employer", "Job_Title", "Monthly_Wage", "Name", "Year", "Gender", "Race")

In [ ]:
head(data, 5)

### Missing Value Visualization

In [ ]:
vis_miss(data) + labs(x = "Missing Value Visualization")

### How is the gender portion?

In [ ]:
ggplot(count(data, Gender), aes(x="", y=n, fill=Gender)) +
  geom_bar(stat="identity", width=1) +
  coord_polar("y", start=0) + theme_void()

### What is the difference between the wages of policemen and policewomen?

In [ ]:
male <- data[data$Gender == 'male', ]
female <- data[data$Gender == 'female', ]

In [ ]:
genders <- c(rep("male" , 2) , rep("female" , 2))
condition <- rep(c("Annual_Wage" , "Monthly_Wage") , 2)
value <- c(colMeans(male[,1], na.rm=TRUE), colMeans(male[,4], na.rm=TRUE), colMeans(female[,1], na.rm=TRUE), colMeans(female[,4], na.rm=TRUE))
wages <- data.frame(genders,condition,value)
 
# Grouped
ggplot(wages, aes(fill=genders, y=value, x=condition)) + 
    geom_bar(position="dodge", stat="identity")

### How differently are the cops paid?

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)
layout(matrix(c(1,1,1,1,1,2,3,4,5,6), 2, 5, byrow = TRUE), heights=c(1,1), widths=c(1,1,1,1,1))
ggplot(data, aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) + 
  ggtitle('Overall')
ggplot(data[data$Employer == 'City of Boston', ], aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) + 
  ggtitle('Boston')
ggplot(data[data$Employer == 'City of Brockton', ], aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) + 
  ggtitle('Brockton')
ggplot(data[data$Employer == 'City of Cambridge', ], aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) +
  ggtitle('Cambridge')
ggplot(data[data$Employer == 'City of Lynn', ], aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) + 
  ggtitle('Lynn')
ggplot(data[data$Employer == 'City of Springfield', ], aes(x=Annual_Wage, fill=Gender, color=Gender)) +
  geom_histogram(position="identity", alpha=0.5, bins=20) + 
  ggtitle('Springfield')

### Ethnic Distribution

In [ ]:
data$Race[data$Race == 'W_NL'] <- 'White'
data$Race[data$Race == 'B_NL'] <- 'Black'
data$Race[data$Race == 'HL'] <- 'Hispanic/Latin'
data$Race[data$Race == 'A'] <- 'Asian'

In [ ]:
library(treemap)
 
# treemap
options(repr.plot.width = 14, repr.plot.height = 8)
treemap(aggregate(data, by=list(data$Gender, data$Race), FUN=length),
            index=c("Group.2","Group.1"),
            vSize="Annual_Wage",
            type="index"
            ) 

#### Salary Difference between Races

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
ggplot(data, aes(x=Race, y=Annual_Wage, fill=Race)) + # fill=name allow to automatically dedicate a color for each group
  geom_violin()

### The difference of Salary in terms of Positions

In [ ]:
data$Job_Title[(data$Job_Title %like% '%Detect%')] <- 'Detective'
data$Job_Title[(data$Job_Title %like% '%Lieut%')] <- 'Lieutenant'
data$Job_Title[(data$Job_Title %like% '%Serg%') | (data$Job_Title %like% '%serg%')] <- 'Sergeant'
data$Job_Title[(data$Job_Title %like% '%Cap%')] <- 'Captain'
data$Job_Title[(data$Job_Title %like% '%Dir%')] <- 'Director'
data$Job_Title[(data$Job_Title %like% '%Commi%')] <- 'Commissioner'
data$Job_Title[(data$Job_Title %like% '%Off%')] <- 'Officer'
data$Job_Title[(data$Job_Title %like% '%Detect%')==FALSE & (data$Job_Title %like% '%Lieut%') == FALSE &(data$Job_Title %like% '%Serg%') == FALSE &
              (data$Job_Title %like% '%serg%') == FALSE & (data$Job_Title %like% '%Cap%') == FALSE & (data$Job_Title %like% '%Dir%') == FALSE & 
              (data$Job_Title %like% '%Commi%') == FALSE & (data$Job_Title %like% '%Off%') == FALSE & is.na(data$Job_Title) == FALSE] <- 'Others'

In [ ]:
unique(data$Job_Title)

#### Imputation

In [ ]:
library(mice)

data <- data %>% mutate(Job_Title = as.factor(Job_Title)) %>% mutate(Monthly_Wage = as.numeric(Monthly_Wage))
init = mice(data, maxit=0) 
meth = init$method
meth[c("Monthly_Wage")]="pmm" 
meth[c("Job_Title")]="polyreg"
set.seed(103)
imputed = mice(data, method=meth, m=50)
data <- complete(imputed)

In [ ]:
gg_miss_var(data)

In [ ]:
data$Job_Title[(data$Job_Title %like% '%Detect%')] <- 'Detective'
data$Job_Title[(data$Job_Title %like% '%Lieut%')] <- 'Lieutenant'
data$Job_Title[(data$Job_Title %like% '%Serg%') | (data$Job_Title %like% '%serg%')] <- 'Sergeant'
data$Job_Title[(data$Job_Title %like% '%Cap%')] <- 'Captain'
data$Job_Title[(data$Job_Title %like% '%Dir%')] <- 'Director'
data$Job_Title[(data$Job_Title %like% '%Commi%')] <- 'Commissioner'
data$Job_Title[(data$Job_Title %like% '%Off%')] <- 'Officer'
data$Job_Title[(data$Job_Title %like% '%Detect%')==FALSE & (data$Job_Title %like% '%Lieut%') == FALSE &(data$Job_Title %like% '%Serg%') == FALSE &
              (data$Job_Title %like% '%serg%') == FALSE & (data$Job_Title %like% '%Cap%') == FALSE & (data$Job_Title %like% '%Dir%') == FALSE & 
              (data$Job_Title %like% '%Commi%') == FALSE & (data$Job_Title %like% '%Off%') == FALSE & is.na(data$Job_Title) == FALSE] <- 'Others'

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(data, aes(x=Job_Title, y=Annual_Wage)) + 
geom_boxplot() + 
ggtitle("Cops' Salaries between Different Positions") +
theme(legend.position="none") +
scale_fill_brewer(palette="Dark2")

### Is there change in salary over years?

In [ ]:
library(hrbrthemes)
library(viridis)

data %>%
  ggplot( aes(x=Year, y=Annual_Wage, fill=Year, group=Year)) +
    geom_boxplot() +
    scale_fill_viridis(discrete = FALSE, alpha=0.6) +
    geom_jitter(color="pink", size=0.4, alpha=0.9) +
    theme_ipsum() +
    theme(
      legend.position="none",
      plot.title = element_text(size=11)
    ) +
    ggtitle("Cops' Salaries over Years") +
    xlab("")

### How differently are the cops paid within the five cities?

In [ ]:
MAcounties <- map_data("county", region='massachusetts')

In [ ]:
cities <- us.cities[(us.cities$name == 'Boston MA') | (us.cities$name == 'Brockton MA') | 
                    (us.cities$name == 'Cambridge MA') | (us.cities$name == 'Lynn MA') | 
                    (us.cities$name == 'Springfield MA'), ]

In [ ]:
data$Employer[(data$Employer == 'City Of Boston') | (data$Employer == 'City of Boston')] <- 'Boston MA'
data$Employer[(data$Employer == 'City Of Brockton') | (data$Employer == 'City of Brockton')] <- 'Brockton MA'
data$Employer[(data$Employer == 'City Of Cambridge') | (data$Employer == 'City of Cambridge')] <- 'Cambridge MA'
data$Employer[(data$Employer == 'City Of Lynn') | (data$Employer == 'City of Lynn')] <- 'Lynn MA'
data$Employer[(data$Employer == 'City Of Springfield') | (data$Employer == 'City of Springfield')] <- 'Springfield MA'

In [ ]:
average <- data %>%
group_by(Employer) %>%
summarize(Mean = mean(Annual_Wage, na.rm=TRUE))

In [ ]:
cities <- inner_join(cities, average, by = c("name"="Employer"))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)
ggplot() + 
  geom_polygon( data=MAcounties, aes(x=long, y=lat, group=group),
                color="black", fill="lightblue" ) + 
  geom_point(data=cities, aes(x=long, y=lat, size = Mean), color = "gold") +
  geom_text(data=cities, aes(x=long, y=lat, label=name), size = 3, hjust=0, vjust=-1) + 
  scale_size(name="Average Wage of 5 cities")

## Modeling

In [ ]:
sapply(data, function(x) sum(is.na(x)))

In [ ]:
data$Monthly_Wage[is.na(data$Monthly_Wage)] <- mean(data$Monthly_Wage, na.rm=TRUE)

In [ ]:
sapply(data, function(x) sum(is.na(x)))

### One-hot Encoding

In [ ]:
for(unique_value in unique(data$Employer)){
data[paste("Employer", unique_value, sep = ".")] <- ifelse(data$Employer == unique_value, 1, 0)
}

for(unique_value in unique(data$Job_Title)){
data[paste("Job.Title", unique_value, sep = ".")] <- ifelse(data$Job_Title == unique_value, 1, 0)
}
for(unique_value in unique(data$Race)){
data[paste("Race", unique_value, sep = ".")] <- ifelse(data$Race == unique_value, 1, 0)
}
data$Gender[data$Gender == 'male'] <- -1
data$Gender[data$Gender == 'female'] <- 1
data$Gender <- as.numeric(data$Gender)

In [ ]:
drops <- c("Name", "Employer", "Job_Title", "Race")
data <- data[ , !(names(data) %in% drops)]

In [ ]:
install.packages("heatmaply")
library(heatmaply)

In [ ]:
y_features <- c("Annual_Wage", "Monthly_Wage")
X <- data[ , !(names(data) %in% y_features)]
Y <- data[ , (names(data) %in% y_features)]

### Standardization

In [ ]:
X <- X %>% mutate_at(c("Year"), ~(scale(.) %>% as.vector))

### PCA

In [ ]:
# before PCA
heatmaply_cor(
  cor(X),
  xlab = "Features", 
  ylab = "Features"
)

In [ ]:
X.pca <- prcomp(X, center = TRUE,scale. = TRUE)
options(repr.plot.width = 6, repr.plot.height = 6)
cumpro <- cumsum(X.pca$sdev^2 / sum(X.pca$sdev^2))
plot(cumpro[0:15], xlab = "PC #", ylab = "Amount of explained variance", main = "Cumulative variance plot")
abline(v = 15, col="blue", lty=5)
abline(h = 0.95, col="blue", lty=5)
legend("topleft", legend=c("Cut-off @ PC15"),
       col=c("blue"), lty=5, cex=0.6)

In [ ]:
sample(X.pca)

In [ ]:
X.pca <- predict(X.pca, newdata = X)[, 1:15]

In [ ]:
heatmaply_cor(
  cor(X.pca),
  xlab = "PCs", 
  ylab = "PCs"
)

### Models

In [ ]:
library(caret)

In [ ]:
summary(
    train(
      Annual_Wage ~ ., 
      Y,
      method = "lm",
      trControl = trainControl(
        method = "cv", 
        number = 10,
        verboseIter = FALSE
      )
    )
        )

#### Linear Regression

In [ ]:
data<-cbind(Y$Monthly_Wage, X.pca)
# colnames(data)[1] <- "Annual_Wage"
colnames(data)[1] <- "Monthly_Wage"

In [ ]:
summary(
    train(
      Monthly_Wage ~ ., 
      data,
      method = "lm",
      trControl = trainControl(
        method = "cv", 
        number = 10,
        verboseIter = FALSE
      )
    )
        )

#### Logistics Regression

In [ ]:
summary(
    train(
      Monthly_Wage ~ ., 
      data,
      method = "glm",
      trControl = trainControl(
        method = "cv", 
        number = 10,
        verboseIter = FALSE
      )
    )
        )

#### Decision Tree

In [ ]:
library(rpart)
# fit model
# fit <- rpart(Y$Annual_Wage~., data=as.data.frame(X), control=rpart.control(minsplit=5))
# # summarize the fit
# summary(fit)

summary(
    train(
      Monthly_Wage ~ ., 
      data,
      method = "ctree",
      control=rpart.control(minsplit=5),
      trControl = trainControl(
        method = "cv", 
        number = 10,
        verboseIter = FALSE
      )
    )
        )

#### xgBoosting

In [ ]:
library(gbm)
# fit model
fit <- gbm(Monthly_Wage~., data=as.data.frame(data), distribution="gaussian")
# summarize the fit
summary(fit)

#### Random Forest

In [ ]:
# load the package
library(randomForest)
# fit model
fit <- randomForest(Monthly_Wage~., data=as.data.frame(data), verboseIter = FALSE)
# summarize the fit
summary(fit)

#### KFold RMSE Comparison

In [ ]:
kfold.compare <- function(model, data) {
    training.samples <-as.data.frame(data)$Monthly_Wage %>%
    createDataPartition(p = 0.8, list = FALSE)
    train.data  <- data[training.samples, ]
    test.data <- data[-training.samples, ]
      if (model == "lm"){
          predictor <- train(
              Monthly_Wage ~ ., 
              train.data,
              method = "lm",
              trControl = trainControl(
                method = "cv", 
                number = 10,
                verboseIter = FALSE
              )
            )
        predictions <- predictor %>% predict(test.data)
        print(paste0("Linear Regression RMSE:", RMSE(predictions, as.data.frame(test.data)$Monthly_Wage)))
      }
          
      else if (model == "glm") {
          predictor <- train(
              Monthly_Wage ~ ., 
              train.data,
              method = "glm",
              trControl = trainControl(
                method = "cv", 
                number = 10,
                verboseIter = FALSE
              )
            )
        predictions <- predictor %>% predict(test.data)
        print(paste0("Logistic Regression RMSE: ", RMSE(predictions, as.data.frame(test.data)$Monthly_Wage)))
      }
            
      else if (model == "dt"){
        predictor <- train(
              Monthly_Wage ~ ., 
              train.data,
              method = "ctree2",
              trControl = trainControl(
                method = "cv", 
                number = 10,
                verboseIter = FALSE
              )
            )
        predictions <- predictor %>% predict(as.data.frame(test.data))
        print(paste0("Decision Tree Regression RMSE: ", RMSE(predictions, as.data.frame(test.data)$Monthly_Wage)))
      }
            
      else if (model == "xgb"){
        predictor <- train(
              Monthly_Wage ~ ., 
              train.data,
              method = "gbm",
#               verboseIter = FALSE,
              trControl = trainControl(
                method = "cv", 
                number = 10,
                verboseIter = FALSE
              )
            )
        predictions <- predictor %>% predict(as.data.frame(test.data), n.trees=10)
        print(paste0("XGBoosting Regression RMSE: ", RMSE(predictions, as.data.frame(test.data)$Monthly_Wage)))
      }
            
      else if (model == "rf"){
        predictor <- randomForest(Monthly_Wage~., data=as.data.frame(train.data))
        predictions <- predictor %>% predict(as.data.frame(test.data))
        print(paste0("Random Forest Regression RMSE: ", RMSE(predictions, as.data.frame(test.data)$Monthly_Wage)))
      }
            
      else {
          print("Invalid model name!")
      }
      
}

In [ ]:
for (model.name in c("lm", "glm", "dt", "rf", "xgb")) {
    kfold.compare(model.name, data)
}